# 练习李老师课程1.3节内容

In [1]:
%jsroot on

In [2]:
!ls -lh treeADC.* //在root终端运行外部命令 !command, 测试发现，此方法在jupyter下有效，在root终端报错！！！

bash: !command,: event not found



In [3]:
TFile *ipf = new TFile("treeADC.root"); //root -l treeADC.root

In [4]:
TCanvas *c1 = new TCanvas();//root环境下可省略
c1->Clear();//root环境下可省略,类似初始化

## TDC分析
- 将多个histogram叠加到一张图上显示的方法，用Draw("same")选项，y轴大小取决于第一个画出的图。

In [5]:
tree->Draw("itu>>htu(420,0,4200)");
TH1D *htu=(TH1D*)gROOT->FindObject("htu");//得到histogram的指针，通过指针进行进一步操作。
tree->Draw("itd>>htd(420,0,4200)");
TH1D *htd=(TH1D*)gROOT->FindObject("htd");
htd->SetLineColor(kGreen);
htu->Draw();
htd->Draw("same");
c1->SetLogy();
c1->Draw();

- 将多个histogram叠加到一张图上显示的方法-II，自动调整每张图的y轴显示范围。

In [6]:
THStack *hs = new THStack("hs","test stacked histograms");
hs->Add(htu);
hs->Add(htd);
hs->Draw("nostack");//替换成hs->Draw();语句，观察效果。无此命令，后续hist会自动归一到第一个hist的坐标轴范围
c1->Draw();

In [7]:
//二维关联图
tree->Draw("itu:itd>>(420,0,4200,420,0,4200)","","colz");//加入pid条件观察图的变化，结合实验设置理解图中不同区域的含义。
gPad->SetLogz();
gPad->SetLogy(0);
c1->Draw();//* 在ROOT环境下可省略

## TDC条件的运用

In [8]:
//位置分布
tree->Draw("itu-itd");
gPad->SetLogy();
c1->Draw();//结果是否正常？如果异常，问题出在哪里？

In [9]:
tree->Draw("itu-itd>>hix(3000,-2000,1000)","itu<4095 && itd<4095");//只有在合理的取值范围的前提下，才能对参数进行运算
gPad->SetLogy(0);
c1->Draw();

## TDC的显示问题

In [10]:
tree->Draw("itu-itd>>(1800,-2000,1000)","itu<4095 && itd<4095");//TDC 道值
gPad->SetLogy(0);
c1->Draw();//显示异常

In [11]:
tree->Draw("tu-td>>(2500,-2000,1000)","itu<4095 && itd<4095");//原始参数
gPad->SetLogy(0);
c1->Draw();

In [12]:
tree->Draw("itu*0.3>>(1200,0,1200)");
c1->Draw();

## 思考题
对比上面5个图，你有什么领悟？
由于实验中ADC获取得到的数据为整型，在Draw时，如果分bin不合适（偶数倍？）就会出现统计错位，而出现人为的凹坑！！！

## QCD分析
- 计算ped峰($V_{base}+V_{ADCbase}$)参数，确定噪声阈值（pedal+3$\sigma_{noise}$）

In [13]:
tree->Draw("iqu>>hquall(420,0,4200)");
TH1D *hquall=(TH1D*)gROOT->FindObject("hquall");
hquall->SetLineColor(kBlack);

tree->Draw("iqu>>hqu1(420,0,4200)","pid==0 || pid==1");
TH1D *hqu1=(TH1D*)gROOT->FindObject("hqu1");
hqu1->SetLineColor(kRed);

tree->Draw("iqu>>hqu2(420,0,4200)","pid==2");
TH1D *hqu2=(TH1D*)gROOT->FindObject("hqu2");
hqu2->SetLineColor(kGreen);


tree->Draw("iqu>>hqu3(420,0,4200)","pid==3");
TH1D *hqu3=(TH1D*)gROOT->FindObject("hqu3");
hqu3->SetLineColor(kBlue);
THStack *hsqu=new THStack("hsqu","qu with different pid");

In [14]:
hsqu->Add(hquall);
hsqu->Add(hqu1);
hsqu->Add(hqu2);
hsqu->Add(hqu3);
hsqu->Draw("nostack");
gPad->SetLogy();
c1->Draw();//辨认pedal峰，超界位置。

In [15]:
tree->Draw("iqd>>hqdall(420,0,4200)");
TH1D *hqdall=(TH1D*)gROOT->FindObject("hqdall");
hqdall->SetLineColor(kBlack);

tree->Draw("iqd>>hqd1(420,0,4200)","pid==0 || pid==1");
TH1D *hqd1=(TH1D*)gROOT->FindObject("hqd1");
hqd1->SetLineColor(kRed);

tree->Draw("iqd>>hqd2(420,0,4200)","pid==2");
TH1D *hqd2=(TH1D*)gROOT->FindObject("hqd2");
hqd2->SetLineColor(kGreen);


tree->Draw("iqd>>hqd3(420,0,4200)","pid==3");
TH1D *hqd3=(TH1D*)gROOT->FindObject("hqd3");
hqu3->SetLineColor(kBlue);
THStack *hsqd=new THStack("hsqd","qd with different pid");

In [16]:
hsqd->Add(hqdall);
hsqd->Add(hqd1);
hsqd->Add(hqd2);
hsqd->Add(hqd3);
hsqd->Draw("nostack");
gPad->SetLogy();
c1->Draw();//辨认pedal峰，超界位置

- 拟合pedo峰，提取拟合参数

In [17]:
hquall->Fit("gaus","","",0,250);
hqdall->Fit("gaus","","",0,250);
c1->Draw();

 FCN=3942.09 FROM MIGRAD    STATUS=CONVERGED      78 CALLS          79 TOTAL
                     EDM=1.86113e-08    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     1.13304e+04   3.58571e+01   8.58443e-01   3.87611e-07
   2  Mean         1.63452e+02   2.33505e-01   5.65227e-03  -7.40369e-04
   3  Sigma        6.34316e+01   1.98794e-01   2.21447e-05   2.21164e-01
 FCN=4294.18 FROM MIGRAD    STATUS=CONVERGED      72 CALLS          73 TOTAL
                     EDM=1.78727e-09    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     1.11862e+04   3.56168e+01   8.69921e-01  -1.42459e-07
   2  Mean         1.55978e+02   2.33246e-01   6.09768e-03   1.53855e-04
   3  Sigma        6.60916e+01   

## 提取拟合结果的方法

In [18]:
TF1 *fgaus[2];
Double_t ped[2],sigma[2];//u,d
TString sq[2]={"qu","qd"};

In [19]:
fgaus[0]=hquall->GetFunction("gaus");//得到拟合函数的指针
fgaus[1]=hqdall->GetFunction("gaus");
for(int i=0;i<2;i++) {
    ped[i]=fgaus[i]->GetParameter(1);//得到拟合函数的第二个参数
    sigma[i]=fgaus[i]->GetParameter(2);
    //TString的格式化输出。用法与printf一致。
    TString ss;
    ss.Form("ped_%s=%.2f, sigma_%s=%.2f",sq[i].Data(),ped[i],sq[i].Data(),sigma[i]); 
    cout<<ss<<endl;
}

ped_qu=163.45, sigma_qu=63.43
ped_qd=155.98, sigma_qd=66.09


In [20]:
tree->Draw("iqu:iqd>>(420,0,4200,420,0,4200)","","colz");
c1->SetLogz();
c1->SetLogy(0);
c1->Draw();//* 在ROOT环境下可省略

## 在tree的Draw(" ")内不能直接输入临时定义的动态数据。

In [21]:
tree->Draw("iqu-ped[0]:iqd-ped[1]","iqd<4000");

Error in <TTreeFormula::Compile>:  Bad numerical expression : "ped[0]"
Info in <TSelectorDraw::AbortProcess>: Variable compilation failed: {iqu-ped[0]:iqd-ped[1],iqd<4000}


## ROOT命令的“ ”内加入动态数据的方法-TString::Form()

In [22]:
//qu,qd减去ped，去掉噪声及超界值，选择合理的取值范围
TString stree,scut;
stree.Form("(iqu-%f):(iqd-%f)>>hsud(200,0,4000,200,0,4000)",ped[0],ped[1]);//在""内加入动态数据的方法
scut.Form("iqu>(%f+3*%f) && iqu<4000 && iqd>(%f+3*%f) && iqd<4000",ped[0],sigma[0],ped[1],sigma[1]);
tree->Draw(stree.Data(),scut.Data(),"colz");
c1->Draw();
cout<<"tree: "<<stree.Data()<<endl;
cout<<"cut: "<<scut.Data()<<endl;

tree: (iqu-163.451860):(iqd-155.977974)>>hsud(200,0,4000,200,0,4000)
cut: iqu>(163.451860+3*63.431595) && iqu<4000 && iqd>(155.977974+3*66.091633) && iqd<4000


## 设置参数的别名：tree->SetAlias()

In [23]:
TString squa,sqda;
squa.Form("iqu-%f",ped[0]);
sqda.Form("iqd-%f",ped[1]);
tree->SetAlias("qua",squa.Data());
tree->SetAlias("qda",sqda.Data());
TCut cpid="pid=2";
tree->Draw("sqrt(qua*qda)",scut.Data());
gPad->SetLogy();
c1->Draw();

In [24]:
TString stcut="itu>0&&itu<4000&&itd>0&&itd<4000";
scut=scut+" && "+stcut;
tree->Draw("itd-itu:log(qua/qda)",scut.Data(),"colz");
c1->SetLogz();
c1->SetLogy(0);
c1->Draw();

In [25]:
tree->Draw("sqrt(qua*qda):(td+itu)/2",scut.Data(),"colz");
c1->SetLogz();
c1->SetLogy(0);
c1->Draw();

In [27]:
!jupyter nbconvert exercise1-3 --to html

[NbConvertApp] Converting notebook exercise1-3.ipynb to html


[NbConvertApp] Writing 1567363 bytes to exercise1-3.html


